In [1]:
import os
import numpy as np
import pandas as pd
import importlib
import sys

# our custom utilities
analysis_base_dir = '/Users/davidhalpern/Documents/GitHub/identifying_causal_subsequent_memory_effects/'
analysis_dir = analysis_base_dir + 'preproc/'
fig_dir = analysis_dir + 'figs/'
data_dir = analysis_base_dir + 'data/'
mri_preproc_dir = analysis_base_dir + 'preproc/'

sys.path.append(analysis_base_dir + 'modules/')
import analysis_utils as au
importlib.reload(au)

%matplotlib inline

In [2]:
# %load omni_directory_setup.py
# base_dir = '/Users/shannon/science/HMM/HMM_MRI/omni_BIDS/'
data_base_dir = '/Volumes/data/HMM/HMM_MRI/omni_BIDS/'
data_base_dir = '/Volumes/Seagate/'


# base_dir = '/Volumes/Rockaway/HMM/HMM_MRI/omni_BIDS/'
bids_raw_dir = data_base_dir + 'bids_raw/'
mriqc_dir = bids_raw_dir + 'mriqc_reports/'
fmriprep_dir = data_base_dir + 'derivatives/fmriprep/'

In [10]:
participants_tsv_path = data_dir + 'participants.tsv'
use_peeps_bids, _, _, _ = au.get_mri_use_list(participants_tsv_path)
print(use_peeps_bids)

['sub-11', 'sub-12', 'sub-15', 'sub-16', 'sub-17', 'sub-19', 'sub-20', 'sub-24', 'sub-25', 'sub-26', 'sub-27', 'sub-28', 'sub-29', 'sub-30', 'sub-31', 'sub-32', 'sub-33', 'sub-34', 'sub-35', 'sub-36', 'sub-37', 'sub-38', 'sub-39', 'sub-40', 'sub-42', 'sub-43', 'sub-44', 'sub-45', 'sub-46', 'sub-47', 'sub-48', 'sub-49', 'sub-50', 'sub-51', 'sub-52', 'sub-53', 'sub-54', 'sub-55', 'sub-56', 'sub-57', 'sub-58', 'sub-59', 'sub-60', 'sub-61', 'sub-62', 'sub-63', 'sub-65', 'sub-67', 'sub-68', 'sub-69', 'sub-70', 'sub-73', 'sub-75', 'sub-76', 'sub-77', 'sub-78', 'sub-79']


In [11]:
allsubs_events = pd.read_csv(os.path.join(data_dir, 'mri_subs_events_long.csv'), index_col=0)
allsubs_events = allsubs_events.query('sub_id == @use_peeps_bids')

In [13]:
trial_word_df = allsubs_events[['lith_word', 'study_block', 'study_block_trial_num']].drop_duplicates()

In [27]:
vox_coord_df = pd.read_csv(os.path.join(analysis_base_dir, 'mri_code', 'roi_info', 'vox_coord_mapper.csv'))
vox_coord_df.head()

/Users/davidhalpern/opt/anaconda3/envs/py37/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,x,y,z,roi_num_schaeffer,roi_vox_num_schaeffer,roi_num_HarvardOxford_lat,hemisphere_HarvOxford,roi_num_HarvardOxford_bilat,roi_name
0,0,0,0,0,0,0,none,0,NaN
1,0,0,1,0,1,0,none,0,NaN
2,0,0,2,0,2,0,none,0,NaN
3,0,0,3,0,3,0,none,0,NaN
4,0,0,4,0,4,0,none,0,NaN


In [28]:
vox_coord_df.roi_name.unique()

array([nan, 'left_insula2', 'left_ATL', 'left_RSC', 'right_Precuneus',
       'right_RSC', 'right_ParietalDorsal', 'right_ParietalMid',
       'right_LO', 'left_LO', 'left_Precuneus', 'left_insula1',
       'left_VWFA', 'left_ParietalDorsal', 'left_PRC', 'left_VLPFCUpper',
       'left_ParietalMid', 'left_VLPFCLower', 'right_PRC', 'right_ATL'],
      dtype=object)

In [13]:
all_vox_dfs = []
for peep in use_peeps_bids:
    print(peep)
    csv_folder = (fmriprep_dir + peep + 
                  '/masks/pulls/single_trial_glms/')
    csv_name = (peep + '_all_rois_all_trials_single_trial_glm_tstat1.csv')
    peep_df = pd.read_csv(csv_folder + csv_name, index_col=0)
    peep_df_merge = peep_df.merge(vox_coord_df, on=['x', 'y', 'z'])
    all_vox_dfs.append(peep_df_merge)

sub-11
sub-12
sub-15
sub-16
sub-17
sub-19
sub-20
sub-24
sub-25
sub-26
sub-27
sub-28
sub-29
sub-30
sub-31
sub-32
sub-33
sub-34
sub-35
sub-36
sub-37
sub-38
sub-39
sub-40
sub-42
sub-43
sub-44
sub-45
sub-46
sub-47
sub-48
sub-49
sub-50
sub-51
sub-52
sub-53
sub-54
sub-55
sub-56
sub-57
sub-58
sub-59
sub-60
sub-61
sub-62
sub-63
sub-65
sub-67
sub-68
sub-69
sub-70
sub-73
sub-75
sub-76
sub-77
sub-78
sub-79


In [15]:
all_vox_df = pd.concat(all_vox_dfs)

In [16]:
all_vox_df.to_csv(data_base_dir + 'glm_csvs/all_vox_df.csv')

In [ ]:
all_vox_df = pd.read_csv(data_base_dir + 'glm_csvs/all_vox_df.csv')

In [13]:
all_vox_df_counts = all_vox_df.groupby(['x', 'y', 'z']).agg({'sub_id': 'count'}).reset_index()

In [15]:
all_vox_df_counts.to_csv(data_base_dir + 'glm_csvs/all_vox_df_counts.csv')

In [19]:
all_vox_df_counts.merge(vox_coord_df).query('roi_name == "left_PRC"').sub_id.max()

285

In [33]:
all_vox_df['n_sub_runs'] = all_vox_df.groupby(['x', 'y', 'z'])['sub_id'].transform('count')

In [34]:
all_vox_id_df = all_vox_df[['sub_id', 'x', 'y', 'z', 'study_block', 'n_sub_runs']]
all_vox_count_sub_df = all_vox_id_df.pivot_table(index=['x', 'y', 'z', 'n_sub_runs'], 
               columns=['sub_id', 'study_block'],
               aggfunc='size',
               fill_value=0).reset_index()
all_vox_count_sub_df.columns = [au.flatten_col(col) for col in all_vox_count_sub_df.columns]

In [35]:
all_vox_count_sub_df.to_csv(data_base_dir + 'glm_csvs/all_vox_count_sub_df.csv', index=False)

In [25]:
# 285 = n_subs (57) * n_runs (5)
all_common_vox_df = all_vox_df.query('n_sub_runs == 285')

In [29]:
all_common_vox_df = all_common_vox_df.merge(vox_coord_df)

In [27]:
all_common_vox_df.to_csv(data_base_dir + 'glm_csvs/all_common_vox_df.csv')

In [17]:
all_common_vox_df = pd.read_csv(data_base_dir + 'glm_csvs/all_common_vox_df.csv')

In [18]:
all_common_vox_df_schaefer = all_common_vox_df.drop(columns=
                                                    ['roi_vox_num_schaeffer', 'roi_num_HarvardOxford_lat', 'hemisphere_HarvOxford', 'roi_num_HarvardOxford_bilat', 'n_subs', 'n_sub_runs'])

In [19]:
all_common_vox_df_schaefer = all_common_vox_df_schaefer.query('roi_num_schaeffer != 0')

In [20]:
schaefer_grouped_df = all_common_vox_df_schaefer.groupby(['roi_num_schaeffer'])

In [21]:
for roi, roi_df in schaefer_grouped_df:
    print(roi)
    roi_df_long = pd.wide_to_long(roi_df, stubnames='trial', 
                                  i=['sub_id', 'study_block', 'x', 'y', 'z'], 
                                  j='study_block_trial_num', sep='_').reset_index()
    roi_df_long.rename(columns={'trial': 'value'}, inplace=True)
    roi_word_df = roi_df_long.merge(trial_word_df)
    roi_word_df.to_csv(data_base_dir + 'glm_csvs/schaefer_' + str(roi) + '_df.csv')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


## Targeted ROIs

In [31]:
all_common_vox_df_tr = all_common_vox_df.drop(columns=
                                                    ['roi_vox_num_schaeffer', 'roi_num_schaeffer', 'roi_num_HarvardOxford_lat', 'roi_num_HarvardOxford_bilat', 'hemisphere_HarvOxford', 'n_subs', 'n_sub_runs'])

In [32]:
all_common_vox_df_tr

,Unnamed: 0,sub_id,run_num,x,y,z,study_block,trial_1,trial_2,trial_3,...,trial_37,trial_38,trial_39,trial_40,trial_41,trial_42,trial_43,trial_44,trial_45,roi_name
0,255,sub-11,1,10,38,29,1,-0.401227,0.716418,1.236828,...,2.103258,-1.936857,-0.223667,0.247210,-0.731104,0.664774,-0.449243,-0.081772,-0.689542,NaN
1,256,sub-11,2,10,38,29,2,1.943486,1.240047,0.733148,...,0.645257,-1.877230,-1.665885,2.185750,-1.858374,3.489266,-0.025779,0.329568,0.697957,NaN
2,257,sub-11,3,10,38,29,3,3.179660,1.947709,1.269842,...,-2.995267,1.660333,0.231718,-0.894097,0.912831,2.058998,0.617408,-0.897802,-0.341993,NaN
3,258,sub-11,4,10,38,29,4,-0.173626,4.337019,-0.625911,...,0.097999,-0.395246,0.275675,1.704654,-1.438332,1.550726,2.392762,3.025003,0.805959,NaN
4,259,sub-11,5,10,38,29,5,0.697199,-0.989026,-0.602340,...,2.211187,0.054579,-0.765254,-0.100603,2.824291,2.476462,-3.676131,1.302436,0.058503,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20697550,525965,sub-79,1,67,44,35,1,-0.360901,1.174772,-0.927571,...,-2.063418,0.316615,1.890408,1.046538,-0.456372,-0.652401,0.039835,0.740115,-0.053427,NaN
20697551,525966,sub-79,2,67,44,35,2,-1.102426,-0.408461,0.401192,...,-0.499808,-0.090210,1.249043,-2.007866,1.604924,-0.427856,-0.168545,1.123697,-1.077424,NaN
20697552,525967,sub-79,3,67,44,35,3,-0.047056,-0.821963,-0.712297,...,-0.585264,0.214050,0.188594,-0.214823,-0.551512,0.112084,-1.129569,-0.931254,-0.365503,NaN
20697553,525968,sub-79,4,67,44,35,4,-1.316135,1.138090,0.025852,...,1.109982,0.409702,0.088179,0.408403,-1.719163,0.509374,1.443509,-0.066399,-1.741882,NaN


In [33]:
all_common_vox_df_tr.query('roi_name == roi_name', inplace=True) #https://stackoverflow.com/questions/26535563/querying-for-nan-and-other-names-in-pandas

In [34]:
tr_grouped_df = all_common_vox_df_tr.groupby('roi_name')

In [35]:
for roi, roi_df in tr_grouped_df:
    print(roi)
    roi_df_long = pd.wide_to_long(roi_df, stubnames='trial', 
                                  i=['sub_id', 'study_block', 'x', 'y', 'z'], 
                                  j='study_block_trial_num', sep='_').reset_index()
    roi_df_long.rename(columns={'trial': 'value'}, inplace=True)
    roi_word_df = roi_df_long.merge(trial_word_df)
    roi_word_df.to_csv(data_base_dir + 'glm_csvs/tr_' + str(roi) + '_df.csv')

left_ATL
left_LO
left_ParietalDorsal
left_ParietalMid
left_Precuneus
left_RSC
left_VLPFCLower
left_VLPFCUpper
left_VWFA
left_insula1
left_insula2
right_ATL
right_LO
right_ParietalDorsal
right_ParietalMid
right_Precuneus
right_RSC


# Hippocampus

In [37]:
hipp_anat_vox_overlap = pd.read_csv(os.path.join(analysis_base_dir, 'mri_code', 'roi_info', 'hipp_anat_vox_overlap.csv'))
hipp_anat_vox_overlap.rename(columns={'sub_num': 'sub_id'}, inplace=True)
hipp_anat_vox_overlap.head()

,sub_id,roi_name,vox_num,x,y,z,xyz,all_subs_overlap,n_subs_overlap
0,sub-11,Hipp_L_ant_reslicedBin,0,25,47,22,25_47_22,0,16
1,sub-11,Hipp_L_ant_reslicedBin,1,26,47,22,26_47_22,0,53
2,sub-11,Hipp_L_ant_reslicedBin,2,26,47,23,26_47_23,0,56
3,sub-11,Hipp_L_ant_reslicedBin,3,26,48,22,26_48_22,0,52
4,sub-11,Hipp_L_ant_reslicedBin,4,27,47,22,27_47_22,1,57


In [ ]:
all_vox_df.drop(columns=['Unnamed: 0', 'roi_num_schaeffer', 'roi_vox_num_schaeffer', 'roi_num_HarvardOxford_lat', 
                         'hemisphere_HarvOxford', 'roi_num_HarvardOxford_bilat'], inplace=True)

In [11]:
all_hipp_vox_df = hipp_anat_vox_overlap.merge(all_vox_df, how='left', on=['sub_id', 'x', 'y', 'z'])

In [42]:
all_hipp_vox_df = all_hipp_vox_df[~pd.isnull(all_hipp_vox_df['study_block'])]

In [46]:
all_hipp_vox_df.drop(columns=['Unnamed: 0'], inplace=True)

In [48]:
all_hipp_vox_df.to_csv(data_base_dir + 'glm_csvs/all_hipp_vox_df.csv', index=False)

In [41]:
all_hipp_vox_df = pd.read_csv(data_base_dir + 'glm_csvs/all_hipp_vox_df.csv')

In [62]:
all_hipp_vox_df['n_sub_runs'] = all_hipp_vox_df.groupby(['xyz'])['vox_num'].transform('size')

In [67]:
all_hipp_vox_df

,sub_id,roi_name,vox_num,x,y,z,xyz,all_subs_overlap,n_subs_overlap,run_num,...,trial_37,trial_38,trial_39,trial_40,trial_41,trial_42,trial_43,trial_44,trial_45,n_sub_runs
0,sub-11,Hipp_L_ant_reslicedBin,0,25,47,22,25_47_22,0,16,1.0,...,0.874214,0.539428,-0.728494,0.658135,0.072063,0.689223,-1.088831,0.516016,-0.058430,80
1,sub-11,Hipp_L_ant_reslicedBin,0,25,47,22,25_47_22,0,16,2.0,...,-1.049061,0.131824,0.830456,1.330387,-0.523271,-1.473859,-0.256684,-0.272374,1.104212,80
2,sub-11,Hipp_L_ant_reslicedBin,0,25,47,22,25_47_22,0,16,3.0,...,0.881838,0.190086,0.405229,-0.110388,0.235955,0.001634,-0.755425,1.837670,-1.487013,80
3,sub-11,Hipp_L_ant_reslicedBin,0,25,47,22,25_47_22,0,16,4.0,...,-0.113823,-0.140539,-0.445634,0.953414,-1.788479,0.485860,-0.437162,-0.783480,0.717594,80
4,sub-11,Hipp_L_ant_reslicedBin,0,25,47,22,25_47_22,0,16,5.0,...,-0.224543,1.524048,-0.440523,1.254207,0.737558,-0.290253,-0.165993,0.243221,0.562275,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82817,sub-79,Hipp_R_post_reslicedBin,33,52,40,28,52_40_28,0,45,1.0,...,0.495982,-0.437042,0.857513,0.485530,1.076329,-0.311471,0.995259,0.590681,0.907974,225
82818,sub-79,Hipp_R_post_reslicedBin,33,52,40,28,52_40_28,0,45,2.0,...,0.715068,-0.940194,0.845665,-0.270151,-0.388765,0.884733,-0.225969,0.455964,-1.096869,225
82819,sub-79,Hipp_R_post_reslicedBin,33,52,40,28,52_40_28,0,45,3.0,...,-1.738553,-1.088731,0.197717,1.080150,1.230727,-0.643485,-0.241753,-0.726133,0.189159,225
82820,sub-79,Hipp_R_post_reslicedBin,33,52,40,28,52_40_28,0,45,4.0,...,-0.749392,0.874110,0.389884,-0.695529,-0.940843,0.102909,-0.253632,-0.490176,-0.418706,225


In [68]:
all_hipp_vox_df_long = pd.wide_to_long(all_hipp_vox_df, stubnames='trial', 
                                  i=['sub_id', 'roi_name', 'study_block', 
                                     'x', 'y', 'z', 'xyz', 'n_sub_runs'], 
                                  j='study_block_trial_num', sep='_').reset_index()
all_hipp_vox_df_long.rename(columns={'trial': 'value'}, inplace=True)
all_hipp_word_df = all_hipp_vox_df_long.merge(trial_word_df)

In [69]:
all_hipp_word_df

,sub_id,roi_name,study_block,x,y,z,xyz,n_sub_runs,study_block_trial_num,all_subs_overlap,n_subs_overlap,run_num,vox_num,value,lith_word
0,sub-11,Hipp_L_ant_reslicedBin,1.0,25,47,22,25_47_22,80,1,0,16,1.0,0,-0.604598,pastatas
1,sub-11,Hipp_L_ant_reslicedBin,1.0,26,47,22,26_47_22,265,1,0,53,1.0,1,-1.028576,pastatas
2,sub-11,Hipp_L_ant_reslicedBin,1.0,26,47,23,26_47_23,280,1,0,56,1.0,2,-1.459610,pastatas
3,sub-11,Hipp_L_ant_reslicedBin,1.0,26,48,22,26_48_22,260,1,0,52,1.0,3,0.298578,pastatas
4,sub-11,Hipp_L_ant_reslicedBin,1.0,27,47,22,27_47_22,285,1,1,57,1.0,4,-0.574696,pastatas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3725545,sub-79,Hipp_R_post_reslicedBin,5.0,51,39,29,51_39_29,255,45,0,51,5.0,29,0.560981,vaistas
3725546,sub-79,Hipp_R_post_reslicedBin,5.0,51,40,27,51_40_27,265,45,0,53,5.0,30,0.322067,vaistas
3725547,sub-79,Hipp_R_post_reslicedBin,5.0,51,40,28,51_40_28,270,45,0,54,5.0,31,0.320461,vaistas
3725548,sub-79,Hipp_R_post_reslicedBin,5.0,51,40,29,51_40_29,245,45,0,49,5.0,32,0.838842,vaistas


In [70]:
all_hipp_vox_df_long

,sub_id,roi_name,study_block,x,y,z,xyz,n_sub_runs,study_block_trial_num,all_subs_overlap,n_subs_overlap,run_num,vox_num,value
0,sub-11,Hipp_L_ant_reslicedBin,1.0,25,47,22,25_47_22,80,1,0,16,1.0,0,-0.604598
1,sub-11,Hipp_L_ant_reslicedBin,1.0,25,47,22,25_47_22,80,2,0,16,1.0,0,-0.042440
2,sub-11,Hipp_L_ant_reslicedBin,1.0,25,47,22,25_47_22,80,3,0,16,1.0,0,0.300362
3,sub-11,Hipp_L_ant_reslicedBin,1.0,25,47,22,25_47_22,80,4,0,16,1.0,0,0.762034
4,sub-11,Hipp_L_ant_reslicedBin,1.0,25,47,22,25_47_22,80,5,0,16,1.0,0,0.029084
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3725545,sub-79,Hipp_R_post_reslicedBin,5.0,52,40,28,52_40_28,225,41,0,45,5.0,33,-1.816386
3725546,sub-79,Hipp_R_post_reslicedBin,5.0,52,40,28,52_40_28,225,42,0,45,5.0,33,-0.154091
3725547,sub-79,Hipp_R_post_reslicedBin,5.0,52,40,28,52_40_28,225,43,0,45,5.0,33,-0.065865
3725548,sub-79,Hipp_R_post_reslicedBin,5.0,52,40,28,52_40_28,225,44,0,45,5.0,33,-0.948202


In [25]:
all_hipp_vox_df_long.merge(trial_word_df, how='outer')

,sub_id,roi_name,study_block,x,y,z,xyz,n_subs_overlap,study_block_trial_num,run_num,all_subs_overlap,vox_num,value,lith_word
0,sub-11,Hipp_L_ant_reslicedBin,1.0,25,47,22,25_47_22,16,1,1.0,0,0,-0.604598,pastatas
1,sub-11,Hipp_L_ant_reslicedBin,1.0,26,47,22,26_47_22,53,1,1.0,0,1,-1.028576,pastatas
2,sub-11,Hipp_L_ant_reslicedBin,1.0,26,47,23,26_47_23,56,1,1.0,0,2,-1.459610,pastatas
3,sub-11,Hipp_L_ant_reslicedBin,1.0,26,48,22,26_48_22,52,1,1.0,0,3,0.298578,pastatas
4,sub-11,Hipp_L_ant_reslicedBin,1.0,27,47,22,27_47_22,57,1,1.0,1,4,-0.574696,pastatas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3793257,sub-68,Hipp_R_ant_reslicedBin,NaN,47,50,20,47_50_20,25,45,NaN,0,19,NaN,NaN
3793258,sub-68,Hipp_R_ant_reslicedBin,NaN,48,49,20,48_49_20,44,45,NaN,0,25,NaN,NaN
3793259,sub-73,Hipp_R_ant_reslicedBin,NaN,44,49,23,44_49_23,39,45,NaN,0,0,NaN,NaN
3793260,sub-73,Hipp_R_ant_reslicedBin,NaN,44,50,23,44_50_23,15,45,NaN,0,1,NaN,NaN


In [71]:
all_hipp_word_df.to_csv(data_base_dir + 'glm_csvs/all_hipp_df.csv', index=False)